In [1]:
# Specs:
# 4 seed layers : Canopy, Tree Stratum, Understorey, Shrub Layer.
# Must place 3 seed of different layers in each square meter.
# Seeds must be randomly distributed across the whole area.

In [2]:
from enum import Enum 
import numpy as np
import random

def countSeeds(area_list, seed_layers):
    result = dict.fromkeys(seed_layers, 0)
    
    for x in area_list:
        for seed_type in x:
            result[seed_type] += 1
    
    return result

def printDistribution(area_list, seed_layers):
    result = countSeeds(area_list, seed_layers)
    seeds_count = sum(result.values())
    
    for seed_type, seed_nb in result.items():
        print("{} : {} ({}%)".format(seed_type, seed_nb, (seed_nb / seeds_count) * 100))

In [3]:
def decrementSeedNumber(key, seed_type_array):
    seeds_stocks[key] -= 1
   
    if (seeds_stocks[key] <= 0):  # removing from list if no seeds left
        seed_type_array.remove(key)
    
def selectSeedsFromStock(size, seed_type_array):
    selection = []

    # check size of current selection and if seed_type_array not empty
    while len(selection) < size and seed_type_array:
        size_to_complete = size - len(selection)
        size_possible = min(size_to_complete, len(seed_type_array))
        my_selection = np.random.choice(seed_type_array, size=size_possible, replace=False)
        
        # merge in selection array
        selection = np.concatenate((selection, my_selection), axis=None) 
        
        for seedName in my_selection:
            decrementSeedNumber(seedName, seed_type_array)
        
    return selection

In [4]:
def plantSeeds(area_size, seeds_stocks):
    area = []
    seed_layers = list(seeds_stocks.keys())
    
    for i in range(area_size) :
        area.append(selectSeedsFromStock(3, seed_layers))

    # Shuffle the area to avoid lack of one seed layer at the end of the list.
    # e.g: if there is 20% of one layer the stock will go empty pretty fast 
    # and the distribution won't be uniform accross the area.
    random.shuffle(area)
    return area

In [5]:
def rand_jitter(arr):
    return arr + np.random.uniform(low=0.1, high=0.9, size=len(arr))

def getCoordinates(case_index):
    x = float(case_index % length)
    y = float(case_index // length)
    return x, y

def separateSeeds(linear_area):
    result = {key: {'x' : [], 'y' : []} for key in seeds_stocks.keys()}
    
    for case_index, square in enumerate(linear_area):
        for seed in square:
            x,y = getCoordinates(case_index)
            result[seed]['x'].append(x)
            result[seed]['y'].append(y)
    
    return result

In [6]:
#%matplotlib notebook
np.random.seed(19680801)
import matplotlib.pyplot as plt



def jitter(ax, x, y, s=20, c='b', marker='o', cmap=None, norm=None, vmin=None, vmax=None, alpha=None, linewidths=None, verts=None, **kwargs):
    return ax.scatter(rand_jitter(x), rand_jitter(y), s=s, c=c, marker=marker, cmap=cmap, norm=norm, 
                      vmin=vmin, vmax=vmax, alpha=alpha, linewidths=linewidths, verts=verts, **kwargs)


def getPlot(coordinates_per_seed_type, scale=500, seed_color = {
    'canopy' : 'tab:blue',
    'tree_stratum' : 'tab:orange',
    'understorey' : 'tab:green',
    'shrub_layer' : 'tab:brown'
    }):
    fig, ax = plt.subplots()
    
    for seed_type, coordinates in coordinates_per_seed_type.items():
        marker_letter = seed_type[:1] # first letter
        jitter(ax, coordinates.get('x'), coordinates.get('y'), c=seed_color.get(seed_type), s=scale, label=seed_type,
                   alpha=0.8, 
               #marker="${}$".format(marker_letter)
              )

    ax.legend()
    # Major ticks every 20, minor ticks every 5
    ax.set_xticks(np.arange(0, length + 1, 5))
    ax.set_xticks(np.arange(0, length + 1 , 1), minor=True)
    ax.set_yticks(np.arange(0, width + 1, 5))
    ax.set_yticks(np.arange(0, width + 1 , 1), minor=True)

    ## And a corresponding grid
    ax.grid(which='both')
    ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
    ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    #plt.savefig("test.svg", format="svg")
    #plt.show()
    
    return fig, ax

In [7]:
def disperseSeeds(seeds_stocks, length, width):
    
    area_size = length * width
    seeds_count = area_size * 3

    seed_layers = list(seeds_stocks.keys())
    
    print("area_size : {}".format(area_size))
    print("seeds_count : {}".format(seeds_count))
    
    # validation 
    seed_sum = sum(seeds_stocks.values())

    if (seed_sum != seeds_count) :
        print("The seed sum is {} and must be {}".format(seed_sum, seeds_count))
        return 
    
    area = plantSeeds(area_size, seeds_stocks)
    
    # Need to validate seed number at first since we can lack one seed layer in 
    # the end which will make some square meters fail the "3 different seeds" condition
    # we need to ensure the pourcentage of each seed layer is right
    printDistribution(area, list(seeds_stocks.keys()))
    
    scale = 500/max(length, width)

    coordinates_per_seed_type = separateSeeds(area)
    fig, ax = getPlot(coordinates_per_seed_type, scale=scale)

    return fig, ax

In [13]:
length = 50
width = 30

seeds_stocks = {
'canopy' : 500,
'tree_stratum' : 1400,
'understorey' : 1300,
'shrub_layer' : 1300
}

fig, ax = disperseSeeds(seeds_stocks, length, width)

area_size : 150000
seeds_count : 450000
canopy : 50000 (11.11111111111111%)
tree_stratum : 140000 (31.11111111111111%)
understorey : 130000 (28.888888888888886%)
shrub_layer : 130000 (28.888888888888886%)


In [14]:
fig.savefig("test-png-plot.png", format='png', dpi=1000)

In [12]:
import io

imgdata = io.StringIO()
fig.savefig(imgdata, format='svg')
fig.savefig("test-png-plot.png", format='png', dpi=1000)
imgdata.seek(0)  # rewind the data

svg_data = imgdata.getvalue()  # this is svg data

#save file 
f= open("test-svg-plot.svg","w")
f.write(svg_data)
f.close() 
#file('test.htm', 'w').write(svg_dta)